In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rawankhaled20","key":"f5a17aecb67ca4675b425dadbe4ab60e"}'}

In [3]:
!mkdir -p ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets list

ref                                                                  title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
thedrcat/daigt-v2-train-dataset                                      DAIGT V2 Train Dataset                              29MB  2023-11-16 01:38:36           1441        149  1.0              
muhammadbinimran/housing-price-prediction-data                       Housing Price Prediction Data                      763KB  2023-11-21 17:56:32           6401        119  1.0              
thedevastator/spotify-tracks-genre-dataset                           Spotify Tracks Genre                                 8MB  2023-11-30 04:25:48           1567         44  1.0              
thedevastator/netflix-imdb-scores       

In [6]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 132MB/s]


In [7]:
! unzip twitter-entity-sentiment-analysis.zip

Archive:  twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
pd.options.mode.chained_assignment = None
import os #File location
from wordcloud import WordCloud #Word visualization
import matplotlib.pyplot as plt #Plotting properties
import seaborn as sns #Plotting properties
from sklearn.feature_extraction.text import CountVectorizer #Data transformation
from sklearn.model_selection import train_test_split #Data testing
from sklearn.linear_model import LogisticRegression #Prediction Model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
from sklearn.metrics import accuracy_score, f1_score #Comparison between real and predicted
import re #Regular expressions
import nltk
from nltk import word_tokenize
import time
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#Validation dataset
val=pd.read_csv("twitter_validation.csv", header=None)
#Full dataset for Train-Test
train=pd.read_csv("twitter_training.csv", header=None)

In [3]:
train.columns=['id','information','type','text']
train.head()

,id,information,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [4]:
val.columns=['id','information','type','text']
val.head()

,id,information,type,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [5]:
# Assuming df is your DataFrame and 'label' is the column containing the labels
train = train[train['type'] != 'Irrelevant']
val = val[val['type'] != 'Irrelevant']

In [6]:
train_data=train #[(train["type"] == "Positive") | (train["type"] == "Negative")]
train_data

,id,information,type,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [7]:
val_data=val #[(val["type"] == "Positive") | (val["type"] == "Negative")]
val_data

,id,information,type,text
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...
5,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...
...,...,...,...,...
993,314,Amazon,Negative,Please explain how this is possible! How can t...
994,9701,PlayStation5(PS5),Positive,Good on Sony. As much as I want to see the new...
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [8]:
#Text transformation
train_data["lower"]=train_data.text.str.lower() #lowercase
train_data["lower"]=[str(data) for data in train_data.lower] #converting all to string
train_data["lower"]=train_data.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex
val_data["lower"]=val_data.text.str.lower() #lowercase
val_data["lower"]=[str(data) for data in val_data.lower] #converting all to string
val_data["lower"]=val_data.lower.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex

In [9]:
train_data.head()

,id,information,type,text,lower
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,i am coming to the borders and i will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...


In [10]:
#Count information per category
plot1=train.groupby(by=["information","type"]).count().reset_index()
plot1.head()

,information,type,id,text,lower
0,Amazon,Negative,576,575,576
1,Amazon,Neutral,1236,1207,1236
2,Amazon,Positive,312,308,312
3,ApexLegends,Negative,600,591,600
4,ApexLegends,Neutral,942,936,942


In [11]:
nltk.download('punkt')

#Text splitting
tokens_text = [word_tokenize(str(word)) for word in train_data.lower]
#Unique word counter
tokens_counter = [item for sublist in tokens_text for item in sublist]
print("Number of tokens: ", len(set(tokens_counter)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of tokens:  26396


In [19]:
tokens_text[1]

['i',
 'am',
 'coming',
 'to',
 'the',
 'borders',
 'and',
 'i',
 'will',
 'kill',
 'you',
 'all']

In [20]:
#Choosing english stopwords
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')
stop_words[:5]

['i', 'me', 'my', 'myself', 'we']

In [21]:
#Initial Bag of Words
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    stop_words=stop_words, #English Stopwords
    ngram_range=(1, 1) #analysis of one word
)

In [12]:
#Train - Test splitting
reviews_train, reviews_test = train_test_split(train_data, test_size=0.2, random_state=0)

In [14]:
#Creation of encoding related to train dataset
X_train_bow = bow_counts.fit_transform(reviews_train.lower)
#Transformation of test dataset with train encoding
X_test_bow = bow_counts.transform(reviews_test.lower)

In [15]:
#Labels for train and test encoding
y_train_bow = reviews_train['type']
y_test_bow = reviews_test['type']

In [16]:
#Total of registers per category
y_test_bow.value_counts() / y_test_bow.shape[0]

Negative    0.372478
Positive    0.334306
Neutral     0.293217
Name: type, dtype: float64

In [13]:
#n-gram of 4 words for logistic model only
bow_counts = CountVectorizer(
    tokenizer=word_tokenize,
    ngram_range=(1,4)
)
#Data labeling
X_train_bow = bow_counts.fit_transform(reviews_train.lower)
X_test_bow = bow_counts.transform(reviews_test.lower)
X_val_bow = bow_counts.transform(val_data.lower)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [17]:
# Logistic regression
start=time.time()
model1 = LogisticRegression(C=1, solver="liblinear",multi_class="ovr")
model1.fit(X_train_bow, y_train_bow)
end=time.time()
# Prediction
test_pred = model1.predict(X_test_bow)
tim=end-start
print("Training time: ",tim)
print("Accuracy: ", accuracy_score(y_test_bow, test_pred) * 100)

Training time:  76.63199639320374
Accuracy:  91.91992868141665


In [18]:
# F1 score
f1 = f1_score(y_test_bow, test_pred, average='weighted')
print("F1 Score: ", f1)

F1 Score:  0.9191817967554485


In [19]:
y_val_bow = val_data['type']
Val_pred_2 = model1.predict(X_val_bow)
print("Accuracy: ", accuracy_score(y_val_bow, Val_pred_2) * 100)

Accuracy:  98.91304347826086


In [20]:
import pickle

pickle.dump(model1, open('/content/model_saved2', 'wb'))

In [22]:
test=pd.read_excel("Amazon.xlsx")

In [23]:
test.columns=['product','review','type','link']

# Convert the 'review' column to lowercase
test['review'] = test['review'].str.lower()

# Convert all values in the 'review' column to strings
test['review'] = test['review'].astype(str)

# Remove non-alphanumeric characters using a regular expression
test['review'] = test['review'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

In [24]:
# Convert the 'review' column to lowercase
test['type'] = test['type'].str.lower()

# Convert all values in the 'review' column to strings
test['type'] = test['type'].astype(str)

# Remove non-alphanumeric characters using a regular expression
test['type'] = test['type'].apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))

In [25]:
predictions = []
y_true = test['type']
count = 0
i=0
# Iterate through the 'review' column in the 'test' DataFrame
for review in test['review']:
    # Transform the individual review using the BoW vectorizer
    review_bow = bow_counts.transform([review])

    # Make predictions using model2 for the individual review
    review_prediction = model1.predict(review_bow)
    review_prediction_str = review_prediction[0]  # Get the prediction as a string

    if review_prediction_str.lower() == 'positive':
        review_prediction_str = "positive"
    elif review_prediction_str.lower() == 'negative':
        review_prediction_str = 'negative'
    else:
        review_prediction_str = "neutral"

    # Append the prediction to the list of predictions
    predictions.append(review_prediction_str)

    if review_prediction_str == y_true[i]:
        count += 1
    i+=1

# Print the count and accuracy
print("Correct Predictions:", count)
print("Accuracy:", count / len(predictions))

# Now, 'predictions' contains the predictions for each review in the 'test' DataFrame
print(predictions)

Correct Predictions: 225
Accuracy: 0.5369928400954654
['positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'neutral', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'neutral', 'positive', 'positive', 'neutral', 'neutral', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'positiv

In [26]:
# Filter the DataFrame to keep rows where "prediction" is not equal to "neutral"
df = test[test['type'] != 'neutral']
df = df[test['type'] != 'natural']
df

<ipython-input-26-de82a7ed3a24>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[test['type'] != 'natural']


,product,review,type,link
0,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",good dual dash camera,positive,NaN
1,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",so far so good,positive,NaN
2,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",drains your battery,negative,NaN
3,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",tucson az freezes on hot days multiple issu...,negative,NaN
4,"S1 4K Dash Cam Front and Rear, 1080P Dual GPS ...",don t bother with this buggy camera,negative,NaN
...,...,...,...,...
413,Brooks Men's Ghost 14 Neutral Running Shoe,overall the shoe is quite comfortable to walk ...,negative,NaN
414,Brooks Men's Ghost 14 Neutral Running Shoe,on my third pair of ghost 14 s provided i stre...,positive,NaN
415,Brooks Men's Ghost 14 Neutral Running Shoe,loved these at first but after 9 or ten wearin...,negative,NaN
416,Brooks Men's Ghost 14 Neutral Running Shoe,not really sure what happened with these shoes...,negative,NaN


In [27]:
predictions = []
y_true = df['type']
count = 0
i = 0

# Iterate through the 'review' column in the DataFrame
for review in df['review']:
    # Transform the individual review using the BoW vectorizer
    review_bow = bow_counts.transform([review])

    # Make predictions using model2 for the individual review
    review_prediction = model1.predict(review_bow)
    review_prediction_str = review_prediction[0].lower()  # Convert to lowercase

    # Convert numeric predictions to strings
    if review_prediction_str == 'positive':
        review_prediction_str = 'positive'
    elif review_prediction_str == 'negative':
        review_prediction_str = 'negative'

    # Append the prediction to the list of predictions
    predictions.append(review_prediction_str)

    # Compare the prediction to the true label
    if review_prediction_str == str(y_true.iloc[i]):
        count += 1
    i += 1

# Print the count and accuracy
print("Correct Predictions:", count)
print("Accuracy:", count / len(predictions))

# Now, 'predictions' contains the predictions for each review in the DataFrame
print(predictions)

Correct Predictions: 208
Accuracy: 0.6283987915407855
['positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'neutral', 'positive', 'positive', 'negative', 'negative', 'negative', 'neutral', 'positive', 'positive', 'positive', 'negative', 'neutral', 'negative', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'ne